In [167]:
from torchvision.datasets import ImageFolder
import torch
import torch.nn as nn
import torch.nn.functional
from sklearn.metrics import classification_report
from sklearn import svm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from myDataset import MyDataset

In [158]:
train_path = r'./datasets/train'
test_path = r'./datasets/test'

In [159]:
train_datasets = MyDataset(train_path,80).getDatasets()
test_datasets = MyDataset(test_path,80).getDatasets()

In [160]:
class AutoEncoder(nn.Module):
    def __init__(self, in_dim, hidden_size, out_dim):
        super(AutoEncoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(in_features=in_dim, out_features=hidden_size),
            nn.ReLU()
           
        )
        self.decoder = nn.Sequential(
            nn.Linear(in_features=hidden_size, out_features=out_dim),
            nn.Sigmoid()
        )

    def forward(self, x):
        encoder_out = self.encoder(x)
        decoder_out = self.decoder(encoder_out)
        return encoder_out, decoder_out

In [161]:
autoEncoder = torch.load('sae_modal.pt')

In [162]:
train_datasets_loader = torch.utils.data.DataLoader(train_datasets)
test_datasets_loader = torch.utils.data.DataLoader(test_datasets)

In [163]:
def loader_to_numpy(datasets_loader):
    # 将DataLoader加载形式的数据转换成 numpy的形式，
    # 便于后面svm的训练
    #
    data_list = []
    label_list = []
    for data, label in datasets_loader:
        data_ = data.view(1,-1)
        encoder, decoder = autoEncoder(data_)
        
        data_list.append(encoder.detach().numpy().tolist())
        label_list.append(label)
        
    data_np = np.array(data_list).reshape((len(data_list),-1))
    label_np = np.array(label_list).reshape((len(label_list),-1))

    return data_np, label_np

In [249]:
train_data,train_label = loader_to_numpy(train_datasets_loader)
#创建svm, 核函数为高斯函数
classifier = svm.SVC(gamma=0.005)
#训练svm
classifier.fit(data,label)

SVC(gamma=0.005)

In [250]:
train_data[0].reshape((1,-1)).shape

(1, 120)

In [251]:
classifier.predict(train_data[0].reshape((1,-1)))

array([0], dtype=int64)

# 测试集评价模型

In [252]:
train_predict = classifier.predict(train_data)
print(classification_report(train_predict,train_label))

              precision    recall  f1-score   support

           0       1.00      0.96      0.98      1071
           1       0.92      0.99      0.95       524

    accuracy                           0.97      1595
   macro avg       0.96      0.98      0.97      1595
weighted avg       0.97      0.97      0.97      1595



In [253]:
#转换数据
test_data, test_label = loader_to_numpy(test_datasets_loader)
#训练好的svm，对测试集进行预测
test_predict = classifier.predict(test_data)

In [254]:
print(classification_report(test_predict,test_label))

              precision    recall  f1-score   support

           0       0.96      0.81      0.88       594
           1       0.50      0.86      0.63       131

    accuracy                           0.82       725
   macro avg       0.73      0.84      0.76       725
weighted avg       0.88      0.82      0.84       725

